# Advance Panel Data Methods

In this notebook, we look into additional panel data models and methods. We start with the widely used fixed effects (FE) estimator, followed by random effects (RE). The dummy variable regression and correlated random effects approaches can be used as alternatives and generalizations of FE. Finally, we cover robust formulas for the variance-covariance matrix and the implied "clustered" standard errors.

**Topics:**
1. Fixed Effects Estimation
2. Random Effects Models
3. Dummy Variable Regression and Correlated Random Effects
4. Robust (Clustered) Standard Errors

## 1. Fixed Effects Estimation

We start from the same basic unobserved effects models. Instead of first differencing, we get ride of the unobserved individual effect $a_i$ using the within transfromation:

$$y_{it} = \beta_0 + \beta_1 x_{it1} + ... + \beta_k x_{itk} + a_i + u_{it}$$

$$\bar{y}_{i} = \beta_0 + \beta_1 \bar{x}_{i1} + ... + \beta_k \bar{x}_{ik} + a_i + \bar{u}_{i}$$

$$\ddot{y}_{it} = y_{it} - \bar{y}_{i} = \beta_1 \ddot{x}_{it1} + ... + \beta_k \ddot{x}_{itk} + \ddot{u}_{it}$$

$$t = 1, 2, 3, ..., T$$

$$i = 1, 2, 3, ..., n$$

where $\bar{y}_i$ is the average of $y_{it}$ over time for cross-section unit $i$ and for the other variables accordingly. The within transformation subtracts these individual averages from the respective observations $y_it$.

The **fixed effects (FE)** estimator simply estimates the demeaned equation using pooled OLS. Instead of applying the within transformation to all variables and running **ols**, we can simply use **PanelOLS()** in the module **linearmodels**. Demeaning is considered by adding the word **EntityEffects** to the formula. This has the additional advantage that the degrees of freedom are adjusted to the demeaning and the variance-covariance matrix and standard errors of freedom are adjusted to the demeaning and the variance-covariance matrix and standard errors are adjusted accordingly. We will come back to different ways to get the same estimates. 

### Wooldridge, Example 14.2: Has the Return to Education Changed over Time?

We estimate the change of the return to education over time using a fixed effects estimator. The data set *WAGEPAN* is a balanced panel for *n* = 545 individuals over *T* = 8 years. It includes the index variables **nr** and **year** for indivdiuals and years, respectively. Since **educ** does not change over time, we cannot estimate its overall impact and have to use **drop_absorbed = True** in the estimation. However, we can interact it with time dummies to see how the impact changes over time.

In [1]:
# Import modules
import wooldridge as woo
import pandas as pd
import linearmodels as plm

In [2]:
# Import the 'wagepan' data set
wagepan = woo.dataWoo('wagepan')

# Create new indices with 'nr' and 'year'
wagepan = wagepan.set_index(['nr', 'year'], drop = False)

In [3]:
# FE model estimation
reg = plm.PanelOLS.from_formula(
    formula = 'lwage ~ married +union + C(year)*educ + EntityEffects',
    data = wagepan, drop_absorbed = True)
results = reg.fit()

C:\Users\lokma\Anaconda3\lib\site-packages\linearmodels\panel\model.py:1798: AbsorbingEffectWarning: 
Variables have been fully absorbed and have removed from the regression:

educ

  AbsorbingEffectWarning,


In [4]:
# Print regression table
table = pd.DataFrame({'beta': round(results.params, 4),
                     'se': round(results.std_errors, 4),
                     't-stat': round(results.tstats, 4),
                     'p-value': round(results.pvalues, 4)})

print(f'Fixed Effect Regression Table: \n{table}\n')

Fixed Effect Regression Table: 
                        beta      se   t-stat  p-value
C(year)[1980]         1.3625  0.0162  83.9031   0.0000
C(year)[1981]         1.3400  0.1452   9.2307   0.0000
C(year)[1982]         1.3567  0.1451   9.3481   0.0000
C(year)[1983]         1.3729  0.1452   9.4561   0.0000
C(year)[1984]         1.4468  0.1452   9.9617   0.0000
C(year)[1985]         1.4122  0.1451   9.7315   0.0000
C(year)[1986]         1.4281  0.1451   9.8404   0.0000
C(year)[1987]         1.4529  0.1452  10.0061   0.0000
married               0.0548  0.0184   2.9773   0.0029
union                 0.0830  0.0194   4.2671   0.0000
C(year)[T.1981]:educ  0.0116  0.0123   0.9448   0.3448
C(year)[T.1982]:educ  0.0148  0.0123   1.2061   0.2279
C(year)[T.1983]:educ  0.0171  0.0123   1.3959   0.1628
C(year)[T.1984]:educ  0.0166  0.0123   1.3521   0.1764
C(year)[T.1985]:educ  0.0237  0.0123   1.9316   0.0535
C(year)[T.1986]:educ  0.0274  0.0123   2.2334   0.0256
C(year)[T.1987]:educ  0.0304  0.0

## 2. Random Effects Models

We again base our analysis on the basic unobserved effects model. The **random effect (RE)** model assums that the unobserved effects $a_i$ are independent of (or at least uncorrelated with) the regressors $x_{itj}$ for all *t* and *j* = 1, 2, 3, ..., *k*. Therefore, our main motivation for using **FD** or **FE** disappears: OLS consistently estimates the model parameters under this additional assumption.

However, like the situation with heteroscedasticity and autocorrelation, we can obtain more efficient estimates if we take into account the structure of the variances and covariances of the error term. Wooldridge (2019, Section 14.2) shows that the GLS transformation that takes care of their special structure implied by the **RE** model leads to a quasi-demeaned specification

$$\dot{y}_{it} = y_{it} - \theta \bar{y}_i = \beta_0(1 - \theta) + \beta_1 \dot{x}_{it1} + ... + \beta_k \dot{x}_{itk} + \dot{v}_{it}$$

where $\dot{y}_{it}$ is similar to the demeaned $\ddot{y}_{it}$ presented in the fixed effect equation but subtractsonly a fraction $\theta$ of the individual averages. The same holds for the regressors $x_{itj}$ and the composite error term $v_{it} = a_i + u_{it}$.

The parameter 

$$\theta = 1 - \sqrt{\frac{\sigma_u^2}{\sigma_u^2 + T\sigma_a^2}}$$

depends on the variances of $u_{it}$ and $a_i$and the length of the time series dimention *T*. It is unknown and has to be etimated. Given our experience with **FD** and **FE** estimation, it should not come as a surprise that we can estimate the **RE** model parameters in **linearmodels** using the command **RandomEffects()**. Different versions of estimating the random effects parameter $\theta$ can be implemented and one version is saved as the attribute **theta** in the results object (see the module [documentation](https://bashtage.github.io/linearmodels/panel/panel/linearmodels.panel.model.RandomEffects.html#linearmodels.panel.model.RandomEffects) for more details).

Unlike the **FD** and **FE** estimators, we can include variables in our model that are constant over time for each corss-sectional unit. We can use **pandas** methods to provide a list of these variables as well as of those that do not vary within each point in time.

### Wooldridge, Example 14.4: A Wage Equation Using Pandel Data

The data set *WAGEPAN* was used in the fixed effects estimation. In this example, we first loads the data set and defines the panel structure. Then, we check the panel dimensions and get a list of time-constant variables using **pandas**. Therefore we calculate grouped variances and used the fact that they are zero over time or individual. With these preparations, we get estimates using OLS, RE, and FE estimatiors. We use **PooledOLS()**, **RandomEffects()**, and **PanelOLS()** (with the option **EntityEffects**), respectively.

In [5]:
# Import modules
import wooldridge as woo
import pandas as pd
import linearmodels as plm

In [6]:
# Import the 'wagepan' data set
wagepan = woo.dataWoo('wagepan')

In [7]:
# Print relevant dimension for panel
N = wagepan.shape[0]
T = wagepan['year'].drop_duplicates().shape[0]
n = wagepan['nr'].drop_duplicates().shape[0]

print(f'Total Number of Observation: {N}\n')
print(f'Total Number of Time Periods: {T}\n')
print(f'Total Number of Individuals: {n}\n')

Total Number of Observation: 4360

Total Number of Time Periods: 8

Total Number of Individuals: 545



In [8]:
# Check non-varying variables

# (I) across time and within individuals by calculating individual
# specific variance for each variable
isv_nr = (wagepan.groupby('nr').var() == 0)  # True, if variance is zero

# Choose variables where all grouped variance are zero
noVar_nr = isv_nr.all(axis = 0)  # which cols are completely True
print(f'Non-varying Variables over Time: \n{isv_nr.columns[noVar_nr]}\n')

Non-varying Variables over Time: 
Index(['black', 'hisp', 'educ'], dtype='object')



In [9]:
# (II) across individuals within one point in time for each variable
isv_t = (wagepan.groupby('year').var() == 0)
noVar_t = isv_t.all(axis = 0)
print(f'Non-varying Variable Across Individual: \n{isv_nr.columns[noVar_t]}\n')

Non-varying Variable Across Individual: 
Index(['d81', 'd82', 'd83', 'd84', 'd85', 'd86', 'd87'], dtype='object')



In [10]:
# Create new indices by 'nr' and 'year'
wagepan = wagepan.set_index(['nr', 'year'], drop = False)

In [11]:
# Estimate by First Difference (FD)
reg_ols = plm.PooledOLS.from_formula(
    formula = 'lwage ~ educ + black + hisp + exper + I(exper**2) +'
    'married + union + C(year)', data = wagepan)
results_ols = reg_ols.fit()

In [12]:
# Estimate by Random Effects Model (RE)
reg_re = plm.RandomEffects.from_formula(
    formula = 'lwage ~ educ + black + hisp + exper + I(exper**2) +'
    'married + union + C(year)', data = wagepan)
results_re = reg_re.fit()

In [13]:
# Estimate by Fixed Effects Model (FE)
reg_fe = plm.PanelOLS.from_formula(
    formula = 'lwage ~ I(exper**2) + married + union + C(year) +'
    'EntityEffects', data = wagepan)
results_fe = reg_fe.fit()

In [14]:
# Print Results

theta_hat = results_re.theta.iloc[0, 0]
print(f'Estimated Theta: {theta_hat}\n')

table_ols = pd.DataFrame({'beta': round(results_ols.params, 4),
                     'se': round(results_ols.std_errors, 4),
                     't-stat': round(results_ols.tstats, 4),
                     'p-value': round(results_ols.pvalues, 4)})

print(f'FD Regression Table: \n{table_ols}\n')

table_re = pd.DataFrame({'beta': round(results_re.params, 4),
                     'se': round(results_re.std_errors, 4),
                     't-stat': round(results_re.tstats, 4),
                     'p-value': round(results_re.pvalues, 4)})

print(f'RE Regression Table: \n{table_re}\n')

table_fe = pd.DataFrame({'beta': round(results_fe.params, 4),
                     'se': round(results_fe.std_errors, 4),
                     't-stat': round(results_fe.tstats, 4),
                     'p-value': round(results_fe.pvalues, 4)})

print(f'FE Regression Table: \n{table_fe}\n')

Estimated Theta: 0.6450593029243452

FD Regression Table: 
                 beta      se   t-stat  p-value
C(year)[1980]  0.0921  0.0783   1.1761   0.2396
C(year)[1981]  0.1504  0.0838   1.7935   0.0730
C(year)[1982]  0.1548  0.0893   1.7335   0.0831
C(year)[1983]  0.1541  0.0944   1.6323   0.1027
C(year)[1984]  0.1825  0.0990   1.8437   0.0653
C(year)[1985]  0.2013  0.1031   1.9523   0.0510
C(year)[1986]  0.2340  0.1068   2.1920   0.0284
C(year)[1987]  0.2659  0.1100   2.4166   0.0157
educ           0.0913  0.0052  17.4419   0.0000
black         -0.1392  0.0236  -5.9049   0.0000
hisp           0.0160  0.0208   0.7703   0.4412
exper          0.0672  0.0137   4.9095   0.0000
I(exper ** 2) -0.0024  0.0008  -2.9413   0.0033
married        0.1083  0.0157   6.8997   0.0000
union          0.1825  0.0172  10.6349   0.0000

RE Regression Table: 
                 beta      se  t-stat  p-value
C(year)[1980]  0.0234  0.1514  0.1546   0.8771
C(year)[1981]  0.0638  0.1601  0.3988   0.6901
C(year)[1

The **RE** estimator needs stronger assuptions to be consistent than the **FE** estimator. On the other hand, it is more efficient if these assumptions hold and we can include time constant regressors. A widely used test of this additional assumption is the **Hausman test**. It is based on the comparison between the **FE** and **RE** parameter estimates. We include an example below, which uses the **FE** and **RE** estimates and implements a **Hausman test** as shown in Wooldridge (2010, Section 10.7.3). The null hypothesis that the RE model is consistent is clearly rejected with sensible significant levels like $\alpha$ = 5% or $\alpha$ = 1%. It also demonstrates that implementing a test on your own is a lot more cumbersome than relying completely on a module's routines.

In [15]:
import scipy.stats as stats
import numpy as np

In [16]:
# Extract the estimated parameters and cov from RE and FE models
b_fe = results_fe.params
b_fe_cov = results_fe.cov
b_re = results_re.params
b_re_cov = results_re.cov

In [17]:
# Hausman Test of FE vs. RE
# (I) Find overlapping coefficients
common_coef = set(results_fe.params.index).intersection(results_re.params.index)

# (II) Calculate differences between FE and RE
b_diff = np.array(results_fe.params[common_coef] - results_re.params[common_coef])
df = len(b_diff)
b_diff.reshape((df, 1))
b_cov_diff = np.array(b_fe_cov.loc[common_coef, common_coef] - 
                      b_re_cov.loc[common_coef, common_coef])
b_cov_diff.reshape((df, df))

# (III) Calculate test statistic
stat = abs(np.transpose(b_diff) @ np.linalg.inv(b_cov_diff) @ b_diff)
pval = 1 - stats.chi2.cdf(stat, df)

print(f'Test Statistic: {stat}\n')
print(f'p-value: {pval}\n')

Test Statistic: 43.427071175613065

p-value: 9.15061385153848e-06



## 3. Dummy Variable Regression and Correlated Random Effects

It turns out that we can get the **FE** parameter estimates in two other ways than the within transformation we used previously. The **dummy variable regression** uses OLS on the original variables instead of the transformed ones. But it add *n* - 1 dummy variable (or *n* dummies and removes the constant), one for each cross-sectional unit *i* = 1, 2, 3, ..., *n*. The simplest (although not computationally most efficient) way to implment this in *Python* is to use the cross-sectional index as another categorical variable.

The third way to get the same results is the **correlated random effects (CRE)** approach. Instead of assuming that the individual effects $a_i$ are independent of the regressors $x_{itj}$, we assume that they only depend on the averages over time $\bar{x}_{ij} = \frac{1}{T} \sum_{t=1}^{T} x_{itj}$ :

$$a_i = \gamma_0 + \gamma_1 \bar{x}_{i1} + ... + \gamma_k \bar{x}_{ik} + r_i$$

$$y_{it} = \beta_0 + \beta_1 x_{it1} + ... + \beta_k x_{itk} + a_i + u_{it}$$

$$y_{it} = \beta_0 + \gamma_0 + \beta_1 x_{it1} + ... + \beta_k x_{itk} + \gamma_1 \bar{x}_{i1} + ... + \gamma_k \bar{x}_{ik} + r_i + u_{it}$$

If $r_i$ is uncorrelated with the regressors, we consistently estimate the parameters of this model using the **RE** estimator. In addition to the orginal regressors, we include their average over time.

We use the *WAGEPAN* data set again to estimate the **FE** parameters using the within transfomration (**reg_we**), the dummy variable approach (**reg_dum**), and the CRE approach (**reg_cre**). We also estimate the **RE** version of this model (**reg_re**). The results confirm that the first three methods deliver exactly the same parameter estimates, while the **RE** estimates differ.

In [18]:
# Import modules
import wooldridge as woo
import pandas as pd
import statsmodels.formula.api as smf
import linearmodels as plm

In [19]:
# Import the 'wagepan' data set
wagepan = woo.dataWoo('wagepan')

In [20]:
# Create the time ('t') and entity ('entity') columns
wagepan['t'] = wagepan['year']
wagepan['entity'] = wagepan['nr']

# Set 'nr' as the index
wagepan = wagepan.set_index(['nr'])

In [21]:
# Include group specific means
wagepan['married_avg'] = wagepan.groupby('nr').mean()['married']
wagepan['union_avg'] = wagepan.groupby('nr').mean()['union']
wagepan = wagepan.set_index(['year'], append = True)

**Estimate FE parameters in 3 different ways**

In [22]:
# 1. Within transformation approach
reg_we = plm.PanelOLS.from_formula(
    formula = 'lwage ~ married + union + C(t)*educ + EntityEffects',
    drop_absorbed = True, data = wagepan)
results_we = reg_we.fit()

C:\Users\lokma\Anaconda3\lib\site-packages\linearmodels\panel\model.py:1798: AbsorbingEffectWarning: 
Variables have been fully absorbed and have removed from the regression:

educ

  AbsorbingEffectWarning,


In [23]:
# 2. Dummy variables approach
reg_dum = smf.ols(
    formula = 'lwage ~ married + union + C(t)*educ + C(entity)',
    data = wagepan)
results_dum = reg_dum.fit()

In [24]:
# 3. CRE approach
reg_cre = plm.RandomEffects.from_formula(
    formula = 'lwage ~ married + union + C(t)*educ + married_avg + union_avg',
    data = wagepan)
results_cre = reg_cre.fit()

In [25]:
# Compare to RE estimate
reg_re = plm.RandomEffects.from_formula(
    formula = 'lwage ~ married + union + C(t)*educ',
    data = wagepan)
results_re = reg_re.fit()

In [26]:
# Print results
var_selection = ['married', 'union', 'C(t)[T.1982]:educ']

table = pd.DataFrame({'Within_Trans': round(results_we.params[var_selection], 4),
                     'Dummy': round(results_dum.params[var_selection], 4),
                     'CRE': round(results_cre.params[var_selection], 4),
                     'Random_Effects': round(results_re.params[var_selection], 4)})

print(f'Compare Estimated Parameters from Different Models: \n{table}\n')

Compare Estimated Parameters from Different Models: 
                   Within_Trans   Dummy     CRE  Random_Effects
married                  0.0548  0.0548  0.0548          0.0773
union                    0.0830  0.0830  0.0830          0.1075
C(t)[T.1982]:educ        0.0148  0.0148  0.0148          0.0143



Given we have estiamted the **CRE** model, it is easy to test the null hypothesis that the **RE** estimatimator is consistent. The additional assumptions needed are $\gamma_1 = ... = \gamma_k = 0$. they can easily be tested using an *F* test or the very similar **Wald test**. As you see, **linearmodels** conveniently provides the routines for these tests. Like the Hausman test, we clearly reject the null hypothesis that the **RE** model is appropriate with a tiny *p* value of about 0.0001.

In [27]:
# RE test as an Wald test on the CRE specific coefficients
wtest = results_cre.wald_test(formula = 'married_avg = union_avg = 0')
print(f'Wald Test Results: \n{wtest}\n')

Wald Test Results: 
Linear Equality Hypothesis Test
H0: Linear equality constraint is valid
Statistic: 19.4058
P-value: 0.0001
Distributed: chi2(2)



Another advantage of the **CRE** approach is that we can add time-constant regressors to the model. Since we cannot control for average values $\bar{x}_{ij}$ for these variables, they have to be uncorrelated with $a_i$ for consistent estimation of *their* coefficients. For the other coefficients of the time-varying variables, we still don't need these additional **RE** assumptions.

In the following example, we estimate another version of the wage equation using the **CRE** approach. The variables **married** and **union** vary over time, so we can control for their between effects. The variables **educ**, **black**, **hisp** do not vary. For a causal interpretation of *their* coefficients, we have to rely on uncorrelatedness with $a_i$. Given $a_i$ includes intellignece and other labor market success factors, this uncorrelatedness is more plausible for some variable (like **gender** or **race**) than for other variables (like **education**).

In [28]:
# Import modules
import wooldridge as woo
import pandas as pd
import statsmodels.formula.api as smf
import linearmodels as plm

In [29]:
# Import the 'wagepan' data set
wagepan = woo.dataWoo('wagepan')

In [30]:
# Create the time ('t') and entity ('entity') columns
wagepan['t'] = wagepan['year']
wagepan['entity'] = wagepan['nr']

# Set 'nr' as the index
wagepan = wagepan.set_index(['nr'])

In [31]:
# Include group specific means
wagepan['married_avg'] = wagepan.groupby('nr').mean()['married']
wagepan['union_avg'] = wagepan.groupby('nr').mean()['union']
wagepan = wagepan.set_index(['year'], append = True)

In [32]:
# Estimate CRE parameters
reg = plm.RandomEffects.from_formula(
    formula = 'lwage ~ married + union + educ + '
    'black + hisp + married_avg + union_avg',
    data = wagepan)
results = reg.fit()

In [33]:
# Print regression table
table = pd.DataFrame({'beta': round(results.params, 4),
                     'se': round(results.std_errors, 4),
                     't-stat': round(results.tstats, 4),
                     'p-value': round(results.pvalues, 4)})

print(f'CRE Regression Table: \n{table}\n')

CRE Regression Table: 
               beta      se   t-stat  p-value
married      0.2417  0.0177  13.6772   0.0000
union        0.0700  0.0207   3.3804   0.0007
educ         0.1257  0.0023  55.4837   0.0000
black       -0.0892  0.0499  -1.7864   0.0741
hisp         0.0784  0.0426   1.8428   0.0654
married_avg -0.0436  0.0450  -0.9685   0.3329
union_avg    0.2105  0.0519   4.0576   0.0001



## 4. Robust (Clustered) Standard Errors

We argued above that under the **RE** assumptions, OLS is inefficient but consistent. Instead of using **RE**, we could simply use OLS but would have to adjust the standard errors for the fact that the composite error term $v_{it} = a_i + u_{it}$ is correlated over time because of the constant individual effect $a_i$. In fact, the variance-covariance matrix could be more complex than the **RE** assumption with i.i.d. $u_{it}$ implies. These error terms could be serially correlated and / or heteroscedastic. This would invalidate the standard errors not only of OLS but also of **FD**, **FE**, **RE**, and **CRE**.

There is an elegant solution, especially in panels with a large cross-sectional dimension. Similar to standard errors that are robust with respect to heteroscedasticity in cross-sectional data and serial correlation in time series, there are formulas for the variance-covariance matrix for panel data that are robust with respect to heteroscedasticity and *arbitrary* correlations of the error term within a cross-sectional unit (or "cluster").

These "clustered" standard errors are mentioned in Wooldridge (2019, Section 14.4 and Example 13.9). Different versions of the clustered variance-covariance matrix can be computed in **linearmodels**. We repeat the **FD** regression from Example 13.9 and reports the adjusted standard errors. Similar to the heteroscedasticity-robust standard errors, there are different versions of formulas for clustered standard errors. We first use the default type (**results_default**), a clustered type without (**results_cluster**) and with a small sample correction (**results_css**). The latter uses ```debiased = Ture``` (default) to adjust the degrees of freedom when estimating the covariance.

In [34]:
# Import modules
import wooldridge as woo
import numpy as np
import linearmodels as plm

In [35]:
# Import the 'crime2' data set
crime4 = woo.dataWoo('crime4')

# Create new indices with county and year
crime4 = crime4.set_index(['county', 'year'], drop = False)

In [36]:
# Estimate FD model
reg = plm.FirstDifferenceOLS.from_formula(
    formula = 'np.log(crmrte) ~ year + d83 + d84 + d85 + d86 + d87 +'
                'lprbarr + lprbconv + lprbpris + lavgsen + lpolpc',
            data = crime4)

In [37]:
# Regression with standard SE
results_default = reg.fit()

In [38]:
# Regression with "clustered" SE
results_cluster = reg.fit(cov_type = 'clustered', cluster_entity = True,
                         debiased =False)

In [39]:
# Regression with "clustered" SE (small-sample correction)
results_css = reg.fit(cov_type = 'clustered', cluster_entity = True)

In [40]:
# Print results
table = pd.DataFrame({'betas': round(results_default.params, 4),
                     'se_default': round(results_default.std_errors, 4),
                     'se_cluster': round(results_cluster.std_errors, 4),
                     'se_css': round(results_css.std_errors, 4)})
print(f'Compare SEs from Different Models: \n{table}\n')

Compare SEs from Different Models: 
           betas  se_default  se_cluster  se_css
year      0.0077      0.0171      0.0136  0.0137
d83      -0.0999      0.0239      0.0219  0.0222
d84      -0.1478      0.0413      0.0356  0.0359
d85      -0.1524      0.0584      0.0505  0.0511
d86      -0.1249      0.0760      0.0624  0.0630
d87      -0.0841      0.0940      0.0773  0.0781
lprbarr  -0.3275      0.0300      0.0556  0.0562
lprbconv -0.2381      0.0182      0.0390  0.0394
lprbpris -0.1650      0.0260      0.0451  0.0456
lavgsen  -0.0218      0.0221      0.0254  0.0257
lpolpc    0.3984      0.0269      0.1014  0.1025

